In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
import pandas as pd

def load_dataset(folder_path):
    images = []
    types = []
    colors = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            name_part = filename.rsplit('.', 1)[0]
            type_and_color = name_part.split('_', 1)

            item_type = type_and_color[0].split()[-1].strip().lower()
            color = type_and_color[1].split()[-1].strip().lower()

            if color != "unknown":
                types.append(item_type)
                colors.append(color)

                img = Image.open(os.path.join(folder_path, filename)).resize((128, 128))
                images.append(np.array(img) / 255.0)

    return np.array(images), types, colors

aritzia_folder = 'aritzia_images'
ami_folder = 'ami_images'
maje_folder = 'maje_images'

aritzia_images, aritzia_types, aritzia_colors = load_dataset(aritzia_folder)
ami_images, ami_types, ami_colors = load_dataset(ami_folder)
maje_images, maje_types, maje_colors = load_dataset(maje_folder)

all_images = np.concatenate([aritzia_images, ami_images, maje_images], axis=0)
all_types = aritzia_types + ami_types + maje_types
all_colors = aritzia_colors + ami_colors + maje_colors

type_encoder = pd.factorize(pd.Series(all_types))
color_encoder = pd.factorize(pd.Series(all_colors))

type_labels = tf.keras.utils.to_categorical(type_encoder[0], num_classes=len(type_encoder[1]))
color_labels = tf.keras.utils.to_categorical(color_encoder[0], num_classes=len(color_encoder[1]))

X_train, X_val, y_type_train, y_type_val, y_color_train, y_color_val = train_test_split(
    all_images, type_labels, color_labels, test_size=0.2, random_state=42
)

print("\nData shapes:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"y_type_train: {y_type_train.shape}")
print(f"y_type_val: {y_type_val.shape}")
print(f"y_color_train: {y_color_train.shape}")
print(f"y_color_val: {y_color_val.shape}")

input_layer = tf.keras.layers.Input(shape=(128, 128, 3))
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
shared_features = tf.keras.layers.Flatten()(x)

type_x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(shared_features)
type_x = tf.keras.layers.Dropout(0.5)(type_x)
type_x = tf.keras.layers.Dense(256, activation='relu')(type_x)
type_output = tf.keras.layers.Dense(len(type_encoder[1]), activation='softmax', name='type_output')(type_x)

color_x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(shared_features)
color_x = tf.keras.layers.Dropout(0.5)(color_x)
color_x = tf.keras.layers.Dense(256, activation='relu')(color_x)
color_output = tf.keras.layers.Dense(len(color_encoder[1]), activation='softmax', name='color_output')(color_x)

model = tf.keras.models.Model(
    inputs=input_layer,
    outputs={'type_output': type_output, 'color_output': color_output}
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'type_output': 'categorical_crossentropy',
        'color_output': 'categorical_crossentropy'
    },
    metrics={
        'type_output': ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
        'color_output': ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    }
)

model.summary()

history = model.fit(
    x=X_train,
    y={
        'type_output': y_type_train,
        'color_output': y_color_train
    },
    validation_data=(X_val, {'type_output': y_type_val, 'color_output': y_color_val}),
    epochs=20,
    batch_size=32
)

model.save('combined_clothing_classifier.keras')
print("Model saved as 'combined_clothing_classifier.keras'")

results = model.evaluate(
    X_val,
    {'type_output': y_type_val, 'color_output': y_color_val}
)

print("\nEvaluation Results:")
for metric, value in zip(model.metrics_names, results):
    print(f"{metric}: {value:.4f}")

y_type_pred = np.argmax(model.predict(X_val)['type_output'], axis=1)
y_color_pred = np.argmax(model.predict(X_val)['color_output'], axis=1)

y_type_true = np.argmax(y_type_val, axis=1)
y_color_true = np.argmax(y_color_val, axis=1)

type_labels_in_val = np.unique(y_type_true)
color_labels_in_val = np.unique(y_color_true)

print("\nType Classification Report:")
print(classification_report(
    y_type_true, 
    y_type_pred, 
    labels=type_labels_in_val, 
    target_names=np.array(type_encoder[1])[type_labels_in_val]
))

print("\nColor Classification Report:")
print(classification_report(
    y_color_true, 
    y_color_pred, 
    labels=color_labels_in_val, 
    target_names=np.array(color_encoder[1])[color_labels_in_val]
))



Data shapes:
X_train: (2156, 128, 128, 3)
X_val: (540, 128, 128, 3)
y_type_train: (2156, 95)
y_type_val: (540, 95)
y_color_train: (2156, 230)
y_color_val: (540, 230)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 126, 126,  │        896 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 63, 63,    │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 61, 61,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 30, 30,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 28, 28,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 14, 14,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 25088)     │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 512)       │ 12,845,568 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │ 12,845,568 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 512)       │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 256)       │    131,328 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    131,328 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ color_output        │ (None, 230)       │     59,110 │ dense_3[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ type_output (Dense) │ (None, 95)        │     24,415 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 26,130,565 (99.68 MB)

 Trainable params: 26,130,565 (99.68 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 13s 170ms/step - color_output_accuracy: 0.2103 - color_output_loss: 4.1268 - color_output_precision_1: 0.4331 - color_output_recall_1: 0.0441 - loss: 14.5878 - type_output_accuracy: 0.1325 - type_output_loss: 3.7079 - type_output_precision: 0.3812 - type_output_recall: 0.0112 - val_color_output_accuracy: 0.2981 - val_color_output_loss: 3.2667 - val_color_output_precision_1: 0.9310 - val_color_output_recall_1: 0.0500 - val_loss: 7.0990 - val_type_output_accuracy: 0.2667 - val_type_output_loss: 3.0005 - val_type_output_precision: 0.8846 - val_type_output_recall: 0.0852
Epoch 2/20
68/68 ━━━━━━━━━━━━━━━━━━━━ 12s 169ms/step - color_output_accuracy: 0.2842 - color_output_loss: 3.2682 - color_output_precision_1: 0.6659 - color_output_recall_1: 0.1260 - loss: 6.9536 - type_output_accuracy: 0.2723 - type_output_loss: 2.8193 - type_output_precision: 0.7166 - type_output_recall: 0.0848 - val_color_output_accuracy: 0.3296 - val_color_output_loss: 2.9538 - val_

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa